In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import torch
from torch.distributions.normal import Normal

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Initial State
        self.s_0 = nn.Parameter(torch.tensor([5.525, 5.1, 5.1,5.1], requires_grad=True)).requires_grad_(True)
        
        self.T = nn.Parameter(torch.tensor(torch.rand(4,4), requires_grad=True)).requires_grad_(True)
        self.mu1 = nn.Parameter(torch.tensor(torch.rand(4), requires_grad=True)).requires_grad_(True)
        
        self.mu2 = nn.Parameter(torch.tensor(torch.rand(4), requires_grad=True)).requires_grad_(True)
        
        self.std1 = nn.Parameter(torch.tensor(torch.rand(4), requires_grad=True)).requires_grad_(True)
#         self.std2 = nn.Parameter(torch.tensor(torch.rand(4), requires_grad=True)).requires_grad_(True)
        

        
        
        

    def forward(self, x,t,pred):

        x = F.softmax((x.clone()),dim=0)
        l=0.0
        std1=torch.exp(self.std1)
        

        x_temp=x.clone()
            
            
            
            
        dist=Normal(self.mu1[0]+self.mu2[0]*0.0, std1[0])
        
        o_1 = torch.exp(dist.log_prob(pred[0]))

        x_temp[0]=o_1*x[0].clone()

        dist=Normal(self.mu1[1]+self.mu2[1]*0.0, std1[1])

        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[1]=o_1.clone()*x[1].clone()

        dist=Normal(self.mu1[2]+self.mu2[2]*0.0, std1[2])
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[2]=o_1*x[2].clone()
        
        dist=Normal(self.mu1[3]+self.mu2[3]*0.0, std1[3])
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[3]=o_1*x[3].clone()
        
#         print(x_temp)
        


        x_temp=x_temp.clip(1e-6,1)
        
        x=x_temp.clone()/torch.sum(x_temp.clone())
        l+=torch.log(torch.sum(x_temp.clone()))# -0.01*state
        
        for i in range(1,t): 

            
            x=torch.matmul(x,F.softmax(self.T,dim=1))
            x_temp=x.clone()
            

            x_temp1=x.clone()
            dist=Normal(self.mu1[0]+self.mu2[0]*i, std1[0])
        
            o_1 = torch.exp(dist.log_prob(pred[i]))

            x_temp[0]=o_1*x[0].clone()

            dist=Normal(self.mu1[1]+self.mu2[1]*i, std1[1])

            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[1]=o_1.clone()*x[1].clone()

            dist=Normal(self.mu1[2]+self.mu2[2]*i, std1[2])
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[2]=o_1*x[2].clone()

            dist=Normal(self.mu1[3]+self.mu2[3]*i, std1[3])
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[3]=o_1*x[3].clone()

            
            
            
#             print(x_temp)
#             print(pred[i])
            x_temp=x_temp.clip(1e-6,1)

            x=x_temp.clone()/torch.sum(x_temp.clone())
#             print(x)
#             print("-----------------------------")
            l+=torch.log(torch.sum(x_temp.clone()))
        return l
    
    def transition(self,x):
        x=torch.matmul(x.to(device),F.softmax(self.T,dim=1))
        x_temp=x.clone()
        return x
   

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
device='cpu'

cuda:0


In [5]:

T_matrix=torch.rand(3, 3).to(device)
T_matrix=torch.tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]], device=device)

# T_matrix[1,1]=8

# T_matrix[2,2]=8

# T_matrix[0,0]=8

T_matrix=T_matrix/torch.sum(T_matrix, 1).reshape(4,1)


s_0=torch.rand(3).to(device)

s_0=torch.tensor([1,0,0])
s_0=s_0/torch.sum(s_0)

O_matrix_mean=torch.tensor([1.5,2.5,3.5,5.5]).to(device)
O_matrix_mean1=torch.tensor([0.1,0.02,0.03,0.04]).to(device)
O_matrix_std=torch.tensor([1.1,1.2,1.3,1.4]).to(device)



print(T_matrix)

print(s_0)

tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]])
tensor([1., 0., 0.])


In [6]:
def simulate_HMM(T_matrix,O_matrix,x,t):
    x=0
    
    
    t=0
    o=[]
#     o_1=torch.matmul(x.to(device),O_matrix.to(device))
# #         print(o_1)
    o.append(Normal(O_matrix_mean[x]+(O_matrix_mean1[x]*torch.tensor(t)),O_matrix_std[x]).sample().to(device))
    t=1
    for i in range(1,1000):
#         print(i)
        x=(torch.distributions.Categorical(T_matrix[x].to(device)).sample())
#         print(x)
        
        
        t+=1
        if x==0:
            o.append(Normal(O_matrix_mean[x]+(O_matrix_mean1[x]*torch.tensor(i)),O_matrix_std[x]).sample().to(device))
        if x==1:
            o.append(Normal(O_matrix_mean[x]+torch.exp(O_matrix_mean1[x]*torch.tensor(i)),O_matrix_std[x]).sample().to(device))
        if x==2:
#                 print(x)
            o.append(Normal(O_matrix_mean[x]+torch.exp(O_matrix_mean1[x]*torch.tensor(i))+torch.sin(torch.pi*torch.tensor(i)/5),O_matrix_std[x]).sample().to(device))
        if x==3:
            o.append(Normal(O_matrix_mean[x]+torch.exp(O_matrix_mean1[x]*torch.tensor(i))+torch.sin(torch.pi*torch.tensor(i)/10),O_matrix_std[x]).sample().to(device))
        t1=t
           
        if (t==50) :
            break
    
    
    
    o=torch.tensor(o).to(device)
    return o,t,t1



In [7]:
o=simulate_HMM(T_matrix,O_matrix_mean,torch.tensor([1.0,0,0.0]),1000)
o

(tensor([ 1.6484, -0.1135, -1.0106,  3.9160,  4.0252,  2.8227,  0.4389,  3.0173,
          4.2565,  2.5863,  5.8434,  4.5323,  4.7540,  0.1898,  2.1699,  5.5111,
          5.4589,  6.1784,  5.9991,  7.8665,  5.0768,  5.4769,  4.5437,  1.9177,
          2.6253,  4.6381,  4.9612,  4.9178,  4.2752,  5.2087,  4.0357,  4.2248,
          7.2253,  3.2777,  5.8517,  2.3273,  6.5220,  5.8863,  5.9047,  3.9018,
          5.2483,  5.6553,  9.1395,  6.7830,  5.4767,  6.0004,  4.9289,  5.0437,
          5.2786,  6.1887]),
 50,
 50)

In [8]:
# def distribution_initalize(o):
#     s=torch.zeros(10)
#     for i in range(0,10):
#         s[i]=torch.sum(o==i)
        
#     return s#/torch.sum(s)
# def distribution_update(dist,o):

#     return dist + distribution_initalize(o)


In [9]:
# f=distribution_initalize(o[0])

def Expectation_diff(o_1):
    return torch.sum(o_1[0])


def abs_diff(e):
    k=0
    for i,e1 in enumerate(e):
        for j,e2 in enumerate(e):
            if j==i+1:
                k+=torch.abs(e2-e1)
    return k


In [10]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import time

start_time = time.time()  
for batch_size in [5]:
    trial =0 
    while trial < 5:
        t=50
        # batch_size=20
        try: 
            pred1=[]
            for i in range(0,batch_size):
                pred1.append(simulate_HMM(T_matrix,O_matrix_mean,s_0,t))
            import torch.optim as optim
            net = Net()
            net.to(device)
            optimizer = optim.Adam(net.parameters(), lr=.01)
            scheduler =lr_scheduler.LinearLR(optimizer,start_factor=1, end_factor=0.001, total_iters=75)
            net.train()
    
            running_loss = 0.0
            o,t,t__1=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
            # dist=distribution_initalize(o)
            for epoch in range(100):  # loop over the dataset multiple times
    
    
    
    
                # zero the parameter gradients
                optimizer.zero_grad()
    
                # forward + backward + optimize
    
    
    
                loss=0.0
    
                for i in range(0,batch_size):
                    optimizer.zero_grad()
                    loss=0.0
    
            #         o,t=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
                    o,t,t_11=pred1[i]
                    pred=torch.zeros((1,t), dtype=torch.int32)
                    pred=o
            #         dist=distribution_update(dist,o)
                    loss-=net(net.s_0,t,pred.to(device))
    
                    l1_lambda = 0.001  # adjust the L1 regularization parameter as needed
                    l1_reg = torch.tensor(0.).to(device)
                    for param in net.parameters():
                        l1_reg += torch.norm(param, 1).to(device) 
              
                    loss=loss
                    loss.backward()
                    optimizer.step()
                scheduler.step()
                    # print statistics
            # print(f'[{trial + 1}] loss: {loss/t}')
    
            # print('Finished Training')
    
            torch.save(pred1, "data_normal\ " + str(trial)+" _ "+str(batch_size)+ "_data.pt")
            torch.save(net.state_dict(),  "Model_normal\ " + str(trial)+" _ "+str(batch_size)+ "_model.pth")
            trial+=1
        except:
            print(f'[{trial + 1}] loss: {loss/t}')
            
            
            pass
end_time = time.time()  # Capture end time after the loop completes

print(f"Total execution time: {end_time - start_time} seconds")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3780\1199379550.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.T = nn.Parameter(torch.tensor(torch.rand(4,4), requires_grad=True)).requires_grad_(True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3780\1199379550.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mu1 = nn.Parameter(torch.tensor(torch.rand(4), requires_grad=True)).requires_grad_(True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3780\1199379550.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTens

Total execution time: 327.87485218048096 seconds
